In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "4"

import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import corner
import batman
import emcee
import time
from multiprocessing import Pool
from IPython.display import display, Math
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import matplotlib.ticker as tck

In [ ]:
def semi_major_axis_in_stellar_radii(semi_major_axis, R_star):
    """
    Converting the semi-major axis (AU) to (Stellar radii)

    Parameters
    ----------
    semi_major_axis : float
        Semi-major axis of the planet in AU.

    R_star : float
        Stellar radius in R_sun.

    Returns
    -------
    a_stellar_radii : float
        Semi-major axis of the planet in stellar radii.
    """
    #1 AU = 215.032 R_Sun

    a_stellar_radii = semi_major_axis * 215.032 / R_star

    return a_stellar_radii

In [ ]:
TIC = 'TIC 66818296' #WASP-17 / TIC 66818296

search_result = lk.search_lightcurve(
    target=TIC,
    mission='TESS', author='SPOC')
print("trying to download")
lc_collection = search_result.download_all()
print("Search done")
print(lc_collection)
lc = lc_collection.stitch(corrector_func=lambda x: x.
                          remove_nans().normalize(unit='unscaled'))
#Sector 12 and sector 
lc = lc.remove_outliers(sigma=10)
lc = lc.remove_nans()

In [ ]:
lc

In [ ]:
lc_f = lc.flatten()
lc_bin = lc_f.bin(time_bin_size = 0.006944444) # Default time is in days
lc_bin = lc_bin.remove_nans()
#0.006944444 day = 10 minutes

t = lc_bin.time.btjd
flux_data = lc_bin.flux
yerrs = lc_bin.flux_err

In [ ]:
fig, ax = plt.subplots(1, figsize=(8,6))
plt.errorbar(lc.time, lc.flux, yerrs, fmt='.', color='Black', ecolor='LightGrey') #Unbinned
plt.errorbar(t, flux_data, yerrs, fmt='.', color='Blue', ecolor='LightGrey') #Binned


plt.ylabel("Normalised flux", size=17)
plt.xlabel("Time - 2457000 (BTJD days)", size=17)

ax.xaxis.set_tick_params(labelsize=17)
ax.yaxis.set_tick_params(labelsize=17)

ax.xaxis.set_minor_locator(tck.AutoMinorLocator())
ax.yaxis.set_minor_locator(tck.AutoMinorLocator())

plt.show()


In [ ]:
filename = "wasp_17b_sector12_and_sector38.h5"
reader = emcee.backends.HDFBackend(filename)
tau = reader.get_autocorr_time()
burnin = int(2 * np.max(tau))
thin = int(0.5 * np.min(tau))
flat_samples = reader.get_chain(discard=burnin, flat=True, thin=thin)

In [ ]:
model_T0 = np.percentile(flat_samples[:, 0], 50)
model_Rp = np.percentile(flat_samples[:, 1], 50)
model_per = np.percentile(flat_samples[:, 2], 50)
model_inc = np.percentile(flat_samples[:, 3], 50)
model_a = np.percentile(flat_samples[:, 4], 50)

prior_Rs = 1.583 #R_sun Southworth et al. 2012

model_params = batman.TransitParams()
model_params.t0 = model_T0                                          #time of inferior conjunction
model_params.per = model_per                                        #orbital period (days)
model_params.rp = model_Rp                #planet radius (in units of stellar radii)
model_params.a = semi_major_axis_in_stellar_radii(model_a, prior_Rs) #semi-major axis (in units of stellar radii))
model_params.inc = model_inc                                     #orbital inclination (in degrees)
model_params.ecc = 0.0                                         #eccentricity
model_params.w = 90.                                          #longitude of periastron (in degrees)
model_params.limb_dark = "quadratic"                          #limb darkening model
model_params.u = [0.21768792859790628, 0.2836292883489928]                          #limb darkening coefficients [u1, u2, u3, u4]

m = batman.TransitModel(model_params, t)                      #initializes model
model_flux = m.light_curve(model_params)

In [ ]:
fig, ax = plt.subplots(1, figsize=(8,6))
plt.errorbar(lc.time, lc.flux, yerrs, fmt='.', color='Black', ecolor='LightGrey') #Unbinned
plt.errorbar(t, flux_data, yerrs, fmt='.', color='Blue', ecolor='LightGrey') #Binned
plt.plot(t, model_flux, color='Red')

plt.ylabel("Normalised flux", size=17)
plt.xlabel("Time - 2457000 (BTJD days)", size=17)

ax.xaxis.set_tick_params(labelsize=17)
ax.yaxis.set_tick_params(labelsize=17)

ax.xaxis.set_minor_locator(tck.AutoMinorLocator())
ax.yaxis.set_minor_locator(tck.AutoMinorLocator())

plt.show()
